# Install the necessary libraries

In [1]:
from ultralytics import YOLO
import os
import shutil
from codecarbon import OfflineEmissionsTracker
import torch
pathInicial=os.getcwd() 
os.chdir("yolov8")
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.12.1+cu113 _CudaDeviceProperties(name='Quadro RTX 4000', major=7, minor=5, total_memory=7982MB, multi_processor_count=36)


# We load the roboflow data of the wheelchair.

In [2]:
if not os.path.exists("wheelchair-detection-1"):
    from roboflow import Roboflow
    rf = Roboflow(api_key="GTFcmyUsIPX2cUOUUP8y")
    project = rf.workspace("2458761304-qq-com").project("wheelchair-detection")
    dataset = project.version(1).download("yolov5")
else:
    print("Good,¡The data is already load!")

Good,¡The data is already load!


# Decide if training with the full train dataset or if reducing dataset

You must select one of the possible methods:

- NONE
- SRS
- DES
- MMS
- NRMD
- RKM
- PHL
- PRD
- FES

You must select a reduction rate, for example if you want a reduction rate of 75%, you have to insert 0.25 in perc, that is to remain only with the 25% of full data.

First, we are going to create a neural network with all the train data and then one with a subset of the 50% of the data.

## Train with full train data

In [3]:
os.chdir(pathInicial)
method = "NONE"
perc=0.25
!python ReductionDatasetRoboflow_yolov8.py --name {method} --perc {perc}

2024-09-11 11:10:51.749187: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 11:10:51.764573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 11:10:51.769040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
You have not selected any method, so you are going to train with the complete training set.
Number Original Files: 463
The training set has a size of:  463


We updated the data yaml, in order to correctly run the training and evaluate the model

In [4]:
os.chdir("yolov8")

import yaml
with open("wheelchair-detection-1" + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [5]:
%%writetemplate wheelchair-detection-1/data.yaml
names:
- people
- peopleWithWheelchair
nc: 2
roboflow:
  license: CC BY 4.0
  project: wheelchair-detection
  url: https://universe.roboflow.com/2458761304-qq-com/wheelchair-detection/dataset/1
  version: 1
  workspace: 2458761304-qq-com
path: wheelchair-detection-1/
test: test/images
train: train/images
val: test/images

#  We train YOLOV8 on our reduced dataset

Descargamos modelo

In [6]:
os.chdir(pathInicial + '/yolov8')
model = YOLO("yolov8m.pt")

Congelamos 10 capas iniciales

In [7]:
freeze = 10
freeze = [f'model.{x}.' for x in range(freeze)]  
for k, v in model.named_parameters(): 
    v.requires_grad = True # train all layers 
    if any(x in k for x in freeze): 
      v.requires_grad = False
    print(k, v.requires_grad)

model.model.0.conv.weight False
model.model.0.bn.weight False
model.model.0.bn.bias False
model.model.1.conv.weight False
model.model.1.bn.weight False
model.model.1.bn.bias False
model.model.2.cv1.conv.weight False
model.model.2.cv1.bn.weight False
model.model.2.cv1.bn.bias False
model.model.2.cv2.conv.weight False
model.model.2.cv2.bn.weight False
model.model.2.cv2.bn.bias False
model.model.2.m.0.cv1.conv.weight False
model.model.2.m.0.cv1.bn.weight False
model.model.2.m.0.cv1.bn.bias False
model.model.2.m.0.cv2.conv.weight False
model.model.2.m.0.cv2.bn.weight False
model.model.2.m.0.cv2.bn.bias False
model.model.2.m.1.cv1.conv.weight False
model.model.2.m.1.cv1.bn.weight False
model.model.2.m.1.cv1.bn.bias False
model.model.2.m.1.cv2.conv.weight False
model.model.2.m.1.cv2.bn.weight False
model.model.2.m.1.cv2.bn.bias False
model.model.3.conv.weight False
model.model.3.bn.weight False
model.model.3.bn.bias False
model.model.4.cv1.conv.weight False
model.model.4.cv1.bn.weight False


Entrenamos el modelo

In [8]:
model.train(data = "wheelchair-detection-1/data.yaml", batch=16, imgsz=640, epochs=100, name ="train_yolov8_roboflow_GAM24_fullTrainSet", seed = 0)

Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=wheelchair-detection-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_yolov8_roboflow_GAM24_fullTrainSet3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fal

  0%|          | 0.00/6.25M [00:00<?, ?B/s]

AMP: checks passed ✅


train: Scanning /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/train/labels... 463 images, 0 backgrounds, 

train: New cache created: /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/train/labels.cache


...

                   all         51        120      0.924      0.859      0.927      0.649



100 epochs completed in 0.333 hours.
Optimizer stripped from runs/detect/train_yolov8_roboflow_GAM24_fullTrainSet3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train_yolov8_roboflow_GAM24_fullTrainSet3/weights/best.pt, 52.0MB

Validating runs/detect/train_yolov8_roboflow_GAM24_fullTrainSet3/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00


                   all         51        120       0.93      0.864      0.929       0.65
                people         27         55      0.935      0.786      0.889      0.524
  peopleWithWheelchair         51         65      0.924      0.941       0.97      0.776
Speed: 1.3ms preprocess, 9.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train_yolov8_roboflow_GAM24_fullTrainSet3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fcaa81b3880>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

Cogemos el mejor modelo de los entrenados y validamos el modelo en el conjunto de test, no hace falta argumentos, ya que la configuración y el dataset son recordados de manera automática

In [10]:
bestmodel = YOLO("runs/detect/train_yolov8_roboflow_GAM24_fullTrainSet/weights/best.pt")

bestmodel.val()

Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/test/labels.cache... 51 images, 0 backgrounds
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00


                   all         51        120       0.93      0.865      0.929      0.652
                people         27         55      0.935      0.789      0.888      0.525
  peopleWithWheelchair         51         65      0.924      0.942       0.97      0.779
Speed: 3.6ms preprocess, 16.1ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fcaee053970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

Make the inference in the video we have choose for the M24 GA of REXASI-PRO

In [11]:
bestmodel.predict("../GAM24_video.mp4",save=True,imgsz=640,conf=0.4)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 17.2ms
video 1/1 (frame 2/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 16.3ms
video 1/1 (frame 3/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 16.4ms
video 1/1 (frame 4/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 1 peopleWithWheelchair, 16.3ms


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'people', 1: 'peopleWithWheelchair'}
 obb: None
 orig_img: array([[[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        ...,
 
        [[14,  4,  0],
         [14,  4,  0],
         [14,  4,  0],
         ...,
         [39, 21, 26],
         [37, 19, 24],
         [37, 19, 24]],
 
        [[19,  9,  4],
         [19,  9,  4],
         [19,  9,  4],
         ...,
         [38, 20, 25],
         [36, 18, 23],
         [36, 18, 23]],
 


## Train with a subset of the 50% of the data

In [12]:
os.chdir(pathInicial)
method = "SRS"
perc=0.5
!python ReductionDatasetRoboflow_yolov8.py --name {method} --perc {perc}

2024-09-11 11:34:32.430785: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 11:34:32.446186: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 11:34:32.450648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Selected Method:  SRS
Selected reduction rate:  0.5
There are 463 images in the path yolov8/wheelchair-detection-1/train
100%|██████████████████████████████████████| 49.7M/49.7M [00:01<00:00, 37.1MB/s]
100%|█████████████████████████████████████████| 463/463 [02:37<00:00,  2.94it/s]
[codecarbon INFO @ 11:37:14] offline tracker init
Estimated emissions:  1.19358408

In [13]:
os.chdir(pathInicial + '/yolov8')
model = YOLO("yolov8m.pt")

freeze = 10
freeze = [f'model.{x}.' for x in range(freeze)]  
for k, v in model.named_parameters(): 
    v.requires_grad = True # train all layers 
    if any(x in k for x in freeze): 
      v.requires_grad = False
    print(k, v.requires_grad)

model.train(data = "wheelchair-detection-1/data.yaml", batch=16, imgsz=640, epochs=100, name ="train_yolov8_roboflow_GAM24_subset", seed = 0)

model.model.0.conv.weight False
model.model.0.bn.weight False
model.model.0.bn.bias False
model.model.1.conv.weight False
model.model.1.bn.weight False
model.model.1.bn.bias False
model.model.2.cv1.conv.weight False
model.model.2.cv1.bn.weight False
model.model.2.cv1.bn.bias False
model.model.2.cv2.conv.weight False
model.model.2.cv2.bn.weight False
model.model.2.cv2.bn.bias False
model.model.2.m.0.cv1.conv.weight False
model.model.2.m.0.cv1.bn.weight False
model.model.2.m.0.cv1.bn.bias False
model.model.2.m.0.cv2.conv.weight False
model.model.2.m.0.cv2.bn.weight False
model.model.2.m.0.cv2.bn.bias False
model.model.2.m.1.cv1.conv.weight False
model.model.2.m.1.cv1.bn.weight False
model.model.2.m.1.cv1.bn.bias False
model.model.2.m.1.cv2.conv.weight False
model.model.2.m.1.cv2.bn.weight False
model.model.2.m.1.cv2.bn.bias False
model.model.3.conv.weight False
model.model.3.bn.weight False
model.model.3.bn.bias False
model.model.4.cv1.conv.weight False
model.model.4.cv1.bn.weight False


train: Scanning /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/train/labels... 232 images, 0 backgrounds, 

train: New cache created: /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/train/labels.cache



val: Scanning /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/test/labels.cache... 51 images, 0 backgrounds


Plotting labels to runs/detect/train_yolov8_roboflow_GAM24_subset/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train_yolov8_roboflow_GAM24_subset
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         51        120      0.947      0.843      0.913      0.637



100 epochs completed in 0.191 hours.
Optimizer stripped from runs/detect/train_yolov8_roboflow_GAM24_subset/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train_yolov8_roboflow_GAM24_subset/weights/best.pt, 52.0MB

Validating runs/detect/train_yolov8_roboflow_GAM24_subset/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00


                   all         51        120      0.947      0.843      0.913      0.637
                people         27         55      0.926      0.764      0.853      0.517
  peopleWithWheelchair         51         65      0.968      0.923      0.974      0.757
Speed: 1.2ms preprocess, 9.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train_yolov8_roboflow_GAM24_subset


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fcb1c05e7c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [14]:
bestmodel = YOLO("runs/detect/train_yolov8_roboflow_GAM24_subset/weights/best.pt")

bestmodel.val()

Ultralytics YOLOv8.2.91 🚀 Python-3.9.19 torch-1.12.1+cu113 CUDA:0 (Quadro RTX 4000, 7982MiB)
Model summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /home/vtoscano/VToscano/GA_M24/yolov8/wheelchair-detection-1/test/labels.cache... 51 images, 0 backgrounds
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00


                   all         51        120      0.946      0.843      0.913      0.639
                people         27         55      0.926      0.764      0.852      0.519
  peopleWithWheelchair         51         65      0.966      0.923      0.974      0.759
Speed: 3.2ms preprocess, 15.2ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fcb4814c2b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [15]:
bestmodel.predict("../GAM24_video.mp4",save=True,imgsz=640,conf=0.4)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 1 peopleWithWheelchair, 16.9ms
video 1/1 (frame 2/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 16.4ms
video 1/1 (frame 3/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 16.4ms
video 1/1 (frame 4/664) /home/vtoscano/VToscano/GA_M24/yolov8/../GAM24_video.mp4: 384x640 4 peoples, 1 peopl

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'people', 1: 'peopleWithWheelchair'}
 obb: None
 orig_img: array([[[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        [[37, 36, 38],
         [37, 36, 38],
         [37, 36, 38],
         ...,
         [31, 30, 32],
         [31, 30, 32],
         [31, 30, 32]],
 
        ...,
 
        [[14,  4,  0],
         [14,  4,  0],
         [14,  4,  0],
         ...,
         [39, 21, 26],
         [37, 19, 24],
         [37, 19, 24]],
 
        [[19,  9,  4],
         [19,  9,  4],
         [19,  9,  4],
         ...,
         [38, 20, 25],
         [36, 18, 23],
         [36, 18, 23]],
 
